# Habit persistence

## Overview

__This notebook displays three models and solutions with capital and endogenously determined long-run risk.  The models are considered in each of the three sections of this notebook and include:__

__1. Adjustment cost__

__2. Permanent income__

__3. Habit persistance__

__The first is an AK model with adjustment costs inspired in part by Eberly and Wong but includes a persistence process for technology.  The second is a version of Friedman's permanent income model, and the third one extends this to include habit persistence preferences motivated by the work of Hansen, Sargent Tallarini.__

*This Notebook shows the solution strategy of the habit persistence model with HST's specification, addressed in "Robust social planner with a single capital stock: notes" ("notes" for reference). The notebook has two sections:*

*1. Consumption responses for the permanent and transitory shock processes for habit persistent preferences*

*2. Robustnes*

*Note: To use this notebook, one needs to install `sympy`,  a Python library for symbolic mathematics that can solve system of equations analytically. We use `sympy` here to solve the system dynamics of the habit persistence model. For more information about `sympy`, visit http://docs.sympy.org/latest/tutorial/index.html.*

## Section 3: Habit Persistance

### Section 3.1: Consumption responses under habit persistence

In this section, we solve the system dynamics of the habit persistence model and plot the consumption responses. First, we setup the parameters, shocks and figure format, and import functions from the module `habit_persistence_code`. The parameters follow Section 2 of the chapter.

In [1]:
#==============================================================================
# Setup
#==============================================================================
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact_manual
from IPython.display import display
%matplotlib inline

# Import module
from habit_persistence_code import *

To compute consumption responses under habit persistence, we essentially conduct the following steps:

1. Calculate parameters and Steady State Values
2. Solve $C^{1}_{t}$ and $U^{1}_{t}$ in terms of $Z^{1}_{t}$ and $Z^{1}_{t+1}$
3. Solve for matrices $\mathbb{L}$ and $\mathbb{J}$ (in this notebook I use $\mathbb{L} = I$ and solve for $\mathbb{J}$)
4. Compute $\mathbb{N}_{11}$ and $\mathbb{N}_{12}$
5. Compute the stable dynamic matrix $\mathbb{A}$
6. Perform the two checks: (1) the matrix multiplication is zero (2) eigenvalues of $\mathbb{A}$ coincide with the weakly stable eigenvalues
7. Add the shocks back to the $X$ evolution equation
8. Compute the consumption response process {$C^{1}_{t} + Y^{1}_{t}$}.

More detailed formulas and instructions can be found in "Solution strategy" section in "notes". Here we provide some equations for reference in the codes:

\begin{equation}
\tag{1}
\label{1}
K^{1}_{t+1} = \exp(\rho - \nu) K^{1}_{t} - \exp(-\nu) C^{1}_{t}
\end{equation}

\begin{equation}
\tag{2}
\label{2}
H^{1}_{t+1} = \exp(-\psi - \nu) H^{1}_{t} - [1 - \exp(-\nu - \psi)] C^{1}_{t} - Y_{t+1} + Y_{t}
\end{equation}

\begin{equation}
\tag{3}
\label{3}
U^{1}_{t} = (1-\alpha) \exp[(\eta - 1)(\bar{u} - \bar{c})] C^{1}_{t} + \alpha \exp[(\eta - 1)(\bar{u} - \bar{h})] H^{1}_{t}
\end{equation}

\begin{equation}
\tag{4}
\label{4}
\exp(-\delta - \psi - \nu + \overline{mh})E[MH^{1}_{t+1} + Y^{1}_{t} - Y^{1}_{t+1} | \mathcal{F}_{t}] = \exp(\overline{mh})MH^{1}_{t} - \alpha \exp[(\eta - 1) \bar{u} - \eta \bar{h}] [(\eta - 1) U^{1}_{t} - \eta H^{1}_{t}]
\end{equation}

\begin{equation}
\tag{5}
\label{5}
(1 - \alpha) \exp[(\eta - 1) \bar{u} - \eta \bar{c}] [(\eta - 1) U^{1}_{t} - \eta C^{1}_{t}] = \exp(-\delta - \nu) E\{\exp(\overline{mk}) MK^{1}_{t+1} - [1 - \exp(-\psi)] \exp(\overline{mh}) MH^{1}_{t+1} | \mathcal{F}_{t}\} + \exp(-\delta - \nu) \{\exp(\overline{mk}) - [1 - \exp(-\psi)] \exp(\overline{mh})\} E(Y^{1}_{t} - Y^{1}_{t+1} | \mathcal{F}_{t})
\end{equation}

\begin{equation}
\tag{6}
\label{6}
\exp(-\delta + \rho - \nu) E[MK^{1}_{t+1} + Y^{1}_{t} - Y^{1}_{t+1} | \mathcal{F}_{t}] = MK^{1}_{t}
\end{equation}

We construct two functions to execute the above steps:
- (for steps 1 through 6) `solve_habit_persistence` 
- (for steps 7 and 8) `habit_persistence_consumption_path`

We first solve the habit persistence model with function `solve_habit_persistence`, and then we calculate the consumption responses by inputting the model solution into function `habit_persistence_consumption_path`. I will later test to combine these two functions with the default parameters into a `class` object. For now we just keep the functions separately.

The parameters that can be changed are in `solve_habit_persistence`:

- $\alpha$: share parameter
- $\psi$: depreciation rate, $0\leq\exp(-\psi)<1$
- $\eta$: elasticity of substitution.

In many cases, we would like to plot out the consumption responses given our choices of parameters. The following code constructs an interactive slider interface such that the users can choose the desired parameters for $\alpha$, $\psi$ and $\eta$. The function `plot_habit_consumption_path` imported from the `habit_persistence_code` module helps to create the plot for the consumption responses.

To use the sliders, first slide and select the desired parameters, and then click on "Run Interact".

In [2]:
interactive_plot = interact_manual(plot_habit_consumption_path, 
                                   α=(0.01,0.99,0.01), 
                                   ψ=(0.1,3.0,0.1), 
                                   η=(0,100,10))

### Section 3.2: Robustness

Next we add in a concern about robustness as in Hansen et al. (1999). Represent:

\begin{equation}
Y^{1}_{t+1} - Y^{1}_{t} = \mathbb{S}_y \cdot X_{t} + \mathbb{F}_y \cdot W_{t+1}
\end{equation}

\begin{equation}
U^{1}_{t} = 
\mathbb{S}_{u} \cdot 
\begin{bmatrix}
K^{1}_{t} \\
H^{1}_{t} \\
X_{t}
\end{bmatrix}
\end{equation}

\begin{equation}
V^{1}_{t} - Y^{1}_{t} = 
\mathbb{S}_{v} \cdot 
\begin{bmatrix}
K^{1}_{t} \\
H^{1}_{t} \\
X_{t}
\end{bmatrix}
+ s_{v}
\end{equation}

where $\mathbb{S}_{u}$ comes from the model solution using `solve_habit_persistence`. Specifically, we do the following:

1. Get $U^{1}_{t}$ from the model solution and express it in terms of $Z$
2. Express the co-state expressions $MK^{1}_{t}$ and $MH^{1}_{t}$ in $U^{1}_{t}$ in terms of the states $K^{1}_{t}$, $H^{1}_{t}$, $X_{t}$
3. Collecting terms to get $U^{1}_{t}$ in terms of $K^{1}_{t}$, $H^{1}_{t}$, $X_{t}$

Next, we compute $\mathbb{S}_{v}$ as in Hansen et al. (2008)

\begin{equation}
(\mathbb{S}_{v})' = 
[1 - \exp(-\delta)] (\mathbb{S}_{u})' + 
\exp(-\delta) [(\mathbb{S}_{v})'\mathbb{A} + 
\begin{bmatrix}
0 & 0 & (\mathbb{S}_y)'
\end{bmatrix}]
\end{equation}

The above procedures can be done through the function `get_Sv`. The function uses the inputs from `solve_habit_persistence` to solve for $\mathbb{S}_{v}$.

We also solve $s_{v}$, 

\begin{equation}
s_{v} = \exp(-\delta)[s_{v} - \frac{\xi}{2}|(\mathbb{S}_{v})'\mathbb{B} + \mathbb{S}_{y})'\mathbb{B}_x|^{2}]
\end{equation}

and the uncertainty price vector 

\begin{equation}
(\mathbb{S}_{v})'\mathbb{B} + F_{y}
\end{equation}

with functions `solve_sv` and `get_SvBFy` respectively. 
Now, we test the example that we used in "notes": $(\alpha, \eta, \psi) = (.1, 100, 1.6)$:

In [3]:
#==============================================================================
# Uncertainty price for alpha = .1, eta = 100, psi = 1.6
#==============================================================================
sens_list = []
J, A, N1, N2, Ct, Ut = solve_habit_persistence(alpha = 0.1, eta = 100, psi = 1.6)
Sv = get_Sv(A, J, N1, N2, Ut)
SvBFy = get_SvBFy(Sv)
SvBFy = [float('%.3g' % x) for x in SvBFy[0]]

print('The uncertainty price vector for ' +\
      'alpha = 0.1, eta = 100, psi = 1.6: {0}'.format(SvBFy))

The uncertainty price vector for alpha = 0.1, eta = 100, psi = 1.6: [0.482, 0.00394]


## References

Hansen, Lars Peter and Thomas J. Sargent. 2008. *Robustness*. Princeton, New Jersey:
Princeton University Press.

Hansen, Lars Peter, William Roberds, and Thomas J. Sargent. 1991. Time Series Im-
plications of Present Value Budget Balance and of Martingale Models of Consumption
and Taxes. *In Rational Expectations Econometrics*, edited by Lars Peter Hansen and
Thomas J. Sargent, chap. 5. Boulder, Colorado: Westview Press.

Hansen, Lars Peter, Thomas Sargent, and Thomas Tallarini. 1999. Robust Permanent
Income and Pricing. *Review of Economic Studies* 66 (4):873-907.

Hansen, Lars Peter, John C. Heaton, and Nan Li. 2008. Consumption Strikes Back?:
Measuring Long Run Risk. *Journal of Political Economy*.